In [1]:
import json
import random
from nltk import pos_tag, word_tokenize
from scipy.spatial import distance
import numpy as np
import math
from nltk.corpus import stopwords
import tqdm
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors
from bert_embedding import BertEmbedding
from scipy import spatial
import mxnet as mx
import re

In [2]:
# Hierarchy of the word to be selected in question.
def hierarchy(a, b, c):
    # a = nn_cnt / b = vb_cnt / c = ad_cnt

    if a == b and b == c:
        return 'noun', 'verb', 'adj'

    if a > b:
        if b > c:
            return 'adj', 'verb', 'noun'
        else:
            return 'verb', 'adj', 'noun'

    elif b >= a:
        if a > c:
            return 'adj', 'noun', 'verb'
        else:
            return 'noun', 'adj', 'verb'

    elif c >= a:
        if a > b:
            return 'verb', 'noun', 'adj'
        else:
            return 'noun', 'verb', 'adj'

In [ ]:
# reformat the file and save it as 'embedding'
#glove2word2vec('counter-fitted-vectors.txt', 'embedding')

In [3]:
# load the vectors in model
model = KeyedVectors.load_word2vec_format('embedding', binary = False)

In [4]:
# get list of english stopwords
stop_words = set(stopwords.words('english'))

In [6]:
# List of POS
# VB verb, base form take, VBD verb, past tense tookVBG verb, gerund/present participle takingVBN verb, past participle takenVBP verb, sing. present, non-3d takeVBZ verb, 3rd perso sing. present takes
# JJ adjective ‘big’ JJR adjective, comparative ‘bigger’ JJS adjective, superlative ‘biggest’


pos = {'noun' : set(['NN','NNS']),
       'verb' : set(['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ' ]),
       'adj' : set(['JJ', 'JJR', 'JJS'])}

nn_cnt = 0
vb_cnt = 0
ad_cnt = 0

# load bert model
bert = BertEmbedding(max_seq_length=65)

failed = 0
total = 0


In [7]:
with open('../SQuAD/train-v2.0(1).json') as json_file:
    data = json.load(json_file)
    for run_nr, block in enumerate(data['data']):
        if run_nr<0:
            continue
        else:
            for paragraph in block['paragraphs']:
                for qa in paragraph['qas']:
                    # to break if nothing in the sentence can be changed
                    error = False
                    total += 1

                    # get a sentence
                    sent = qa['question']
                    # tokenize and POS-tag the sentence
                    tkn_sent = word_tokenize(sent)

                    # pos tag the sentence
                    pos_sent = [(word.lower(), tag) for word, tag in pos_tag(tkn_sent)]
                    #print(pos_sent)

                    # select word to be replace given the hierachy provided:
                    # prioritize grammatical classes by counting the frequency in the instances.
                    pos_order = hierarchy(nn_cnt, vb_cnt, ad_cnt)

                    # tags sorted by priority
                    top = pos[pos_order[0]]
                    mid = pos[pos_order[1]]
                    fin = pos[pos_order[2]]

                    # find fitting POS tags
                    top_col, mid_col, fin_col = [],[],[]
                    for idx, inst in enumerate(pos_sent):
                        # to get into the selection the instance has to be in the embedding dictionary and not a stopword
                        if model.__contains__(inst[0]) and not (inst[0] in stop_words):
                            if inst[1] in top:
                                top_col.append(idx)
                            elif inst[1] in mid:
                                mid_col.append(idx)
                            elif inst[1] in fin:
                                fin_col.append(idx)

                    # pick randomly a word according to hierarchy
                    if len(top_col)>0:
                        chosen_idx = random.choice(top_col)
                    elif len(mid_col)>0:
                        chosen_idx = random.choice(mid_col)
                    elif len(fin_col)>0:
                        chosen_idx = random.choice(fin_col)
                    else:
                        error = True
                        failed += 1
                        #print('ERROR: couldnt replace anything', '\n')

                    if error == False:
                        # chosen word and its POS
                        chosen_word = pos_sent[chosen_idx][0]
                        chosen_pos = pos_sent[chosen_idx][1]

                        # the chosen word/pos pair
                        if chosen_pos in pos['noun']:
                            nn_cnt +=1
                        elif chosen_pos in pos['verb']:
                            vb_cnt +=1
                        elif chosen_pos in pos['adj']:
                            ad_cnt +=1

                        # distance to all other words in the embedding dict
                        omega = 0.9
                        mst_smlr = model.most_similar(chosen_word)
                        new_words = [word for word, score in mst_smlr if score<omega]

                        if len(new_words)==0:
                            failed += 1
                            #print('ERROR: couldnt find good alternatives for ', chosen_word, '\n')
                        else:
                            #print(chosen_word, new_words, '\n')

                            # get all possible alternative sentences
                            sentence_options = [re.sub(r"\s's", " s", " ".join(tkn_sent))]
                            for word in new_words:
                                tkn_sent[chosen_idx] = word
                                sentence_options.append(re.sub(r"\s's", " s", ' '.join(tkn_sent[:])))

                            # calculate bert sentence embeddings
                            results = bert.embedding(sentence_options)
                            
                            # calculate similarity scores between suggested and original sentence
                            similarity = []
                            try:
                                og_wrd_embed = results[0][1][chosen_idx]
                            except IndexError:
                                print("Failed: ", failed)
                                print("Total: ", total)
                                print("total nouns: ", nn_cnt)
                                print("total verbs: ", vb_cnt)
                                print("total adj: ", ad_cnt)


                            for sug_sent in results[1:]:
                                similarity.append(1 - spatial.distance.cosine(og_wrd_embed, sug_sent[1][chosen_idx]))

                            # pick the best new sentence
                            max_value = max(similarity)
                            best_sent_idx = similarity.index(max_value)+1
                            qa['question'] = sentence_options[best_sent_idx][:]


        with open('../SQuAD/data_new2.json', 'w') as outfile:
            json.dump(data, outfile)
            print('saved after block ', run_nr,"/442")
    print("done")
    print("Failed: ", failed)
    print("Total: ", total)
    print("total nouns: ", nn_cnt)
    print("total verbs: ", vb_cnt)
    print("total adj: ", ad_cnt)


FileNotFoundError: [Errno 2] No such file or directory: '../SQuAD/train-v2.0(1).json'